In [1]:
# import external libraries

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import re

from sklearn.metrics import mean_squared_error

%matplotlib inline

C:\Users\Abhishek\Anaconda2\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
%run scripts/features.py

In [3]:
# load training and test data set
training = pd.read_table('./data/BingHackathonTrainingData.txt', sep='\t', header=None)
test = pd.read_table('./data/BingHackathonTestData.txt', sep='\t', header=None)

training = training[range(0,  6)]
test = test[range(0, 6)]

In [4]:
training = training.rename(columns={0: 'record_id', 1: 'topic_id', 2: 'publication_year', 3: 'authors', 4: 'title', 5: 'summary'})
test = test.rename(columns={0: 'record_id', 1: 'topic_id', 2: 'publication_year', 3: 'authors', 4: 'title', 5: 'summary'})

In [5]:
# response variable
y = training.publication_year

In [6]:
from sklearn.cross_validation import train_test_split

In [38]:
X_train, X_test, y_train, y_test = train_test_split(training, y, test_size=0.2, random_state=44)

In [39]:
print X_train.shape, X_test.shape, y_train.shape, y_test.shape

(3598, 6) (900, 6) (3598L,) (900L,)


In [ ]:
## Classification
from sklearn.linear_model import LogisticRegression

ft = FeatureTransformer()
logistic_regression = LogisticRegression(C=10, penalty='l2')
pipeline_log_reg = Pipeline([('ft', ft), ('log_reg', logistic_regression)])

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import xgboost as xgb

from sklearn.pipeline import Pipeline

In [40]:
ft = FeatureTransformer()
linear_regression = LinearRegression()

pipeline_reg = Pipeline([('ft', ft), ('reg', linear_regression)])

In [19]:
ft = FeatureTransformer()
extreme_gradient_boosting_regressor = xgb.XGBRegressor()

pipeline_xgb = Pipeline([('ft', ft), ('xgb', extreme_gradient_boosting_regressor)])

In [41]:
# train a linear regression model

pipeline_reg.fit(X_train, y_train)

Pipeline(steps=[('ft', FeatureTransformer()), ('reg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [20]:
# train a Extreme Gradient Boosting Regression model

pipeline_xgb.fit(X_train, y_train)

Pipeline(steps=[('ft', FeatureTransformer()), ('xgb', XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))])

In [44]:
# predictions

predsTrain_Linear = np.ceil(pipeline_reg.predict(X_train))
predsTest_Linear = np.ceil(pipeline_reg.predict(X_test))

In [21]:
# predictions (XGB)

predsTrain_XGB = pipeline_xgb.predict(X_train)
predsTest_XGB = pipeline_xgb.predict(X_test)

In [45]:
print 'Root Mean Square Error on the training examples for Linear Regression %f ' %(np.sqrt(mean_squared_error(y_train, predsTrain_Linear)))
print 'Root Mean Square Error on the test examples for Linear Regression %f ' %(np.sqrt(mean_squared_error(y_test, predsTest_Linear)))

Root Mean Square Error on the training examples for Linear Regression 3.349396 
Root Mean Square Error on the test examples for Linear Regression 3.423449 


In [22]:
print 'Root Mean Square Error on the training examples for XGBoost %f ' %(np.sqrt(mean_squared_error(y_train, predsTrain_XGB)))
print 'Root Mean Square Error on the test examples for XGBoost %f ' %(np.sqrt(mean_squared_error(y_test, predsTest_XGB)))

Root Mean Square Error on the training examples for XGBoost 3.139397 
Root Mean Square Error on the test examples for XGBoost 3.444580 


In [46]:
## fit on the whole dataset
pipeline_reg.fit(training, y)

Pipeline(steps=[('ft', FeatureTransformer()), ('reg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [47]:
# predictions
preds = np.ceil(pipeline_reg.predict(test))

In [65]:
# Submissions
submission_df = pd.DataFrame({'record_id': test.record_id.values, 'year': preds})

In [66]:
submission_df.to_csv('./submissions/first_regression.csv', sep='\t', index=False)

In [64]:
submission_df.shape

(1125, 2)